### Get a Graph of #indiedev Twitter Community.

1) Unify data already extracted and clean DF unnecesary parameters.

2) Get Users for nodes.

3) Make a RT DF.

4) Export Nodes and Edges csv.

5) Load into Gephi.

6) Import Gephi image.

In [1]:
#Requiered libraries
from collections import Counter
from tweepy import OAuthHandler, API
import tweepy
import time
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Auth keys and tokens.   
ds = pd.read_pickle('dogge_secret')
consumer_key        = ds[0]
consumer_secret     = ds[1]
access_token        = ds[2]
access_token_secret = ds[3]

#Tweepy client
auth_handler = OAuthHandler(consumer_key, consumer_secret)
auth_handler.set_access_token(access_token, access_token_secret)
twitter_client = API(auth_handler)

In [2]:
#The following functions get specific data from "retweet_status" and the status user.

In [12]:
#Get Original RT ID and author ID
def get_RT_status(df_):
    RTO,RTOU,RT_time,RTOU_name = [],[],[],[]
    for row in df_['retweeted_status']:
        try:
            RTO.append(row['id'])
        except:
            RTO.append(None)
        try:
            RTOU.append(row['user']['id'])
        except:
            RTOU.append(None)
        try:
            RT_time.append(row['created_at'])
        except:
            RT_time.append(None)
        try:
            RTOU_name.append(row['screen_name'])
        except:
            RTOU_name.append(None)
    df_['RT_id'] = RTO
    df_['RT_userid'] = RTOU
    df_['RT_time'] = RT_time
    df_['RTOU_name'] = RTOU_name
    return df_

In [13]:
#Get User information
def get_User(df_):
    target = ['created_at','favourites_count','id','lang','location','statuses_count','time_zone','verified','screen_name','name']
    tgt_data = {i:[] for i in target}
    for row in df_['user']:
        for k in target:
            try:
                tgt_data[k].append(row[k])
            except:
                tgt_data[k].append(None)
    for k in target:
        df_['usr_'+str(k)] = tgt_data[k]
    return df_

In [14]:
#clean_col returns the desired columns for the dataframe

In [15]:
#Get rid of extra columns
def clean_col(df_):
    target = [u'id',u'in_reply_to_user_id',u'RT_id',u'created_at',u'favorite_count',u'retweet_count',u'text',
                    u'usr_id',u'usr_created_at',u'usr_lang',u'usr_location',u'usr_statuses_count',u'usr_favourites_count',
                    u'usr_time_zone',u'usr_verified',u'RT_userid',u'RT_time','usr_screen_name','usr_name','RTOU_name']
    df_ = df_[target]
    return df_

In [16]:
#process() gives the parsed final df

In [17]:
def process(df_):
    df_ = get_RT_status(df_)
    df_ = get_User(df_)
    df_ = clean_col(df_)
    return df_

In [18]:
#indiegames pickles from 1 to 4 are a panda DataFrame of Twitter Statuses
#Read them
d1 = pd.read_pickle('indiegames')
d2 = pd.read_pickle('indiegames2')
d3 = pd.read_pickle('indiegames3')
d4 = pd.read_pickle('indiegames4')
#Parse them
dd1 = process(d1)
dd2 = process(d2)
dd3 = process(d3)
dd4 = process(d4)
#Join them
final = dd1.append(dd2)
final = final.append(dd3)
final = final.append(dd4)
final = final.drop_duplicates()
final.to_pickle('ig_final')

In [19]:
#Get a network of followers

In [20]:
def get_followers(user_id):
    users = []
    page_count = 0
    try:
        for user in tweepy.Cursor(twitter_client.followers, id=user_id, count=200).pages():
            page_count += 1
            #print 'Getting page {} for followers'.format(page_count)
            users.extend(user)
    except:
        users = 'private'
    if users != 'private':
        users_id = [a.id for a in users]
        return users_id
    else:
        return users

In [21]:
sample_users = list(set(list(set(final.usr_id))+list(set(final.RT_userid))))

In [22]:
clean_sample = [str(i) for i in sample_users if str(i) != 'nan']

In [ ]:
follower_id_ids = {}
for usr in clean_sample[:20]:
    if usr not in follower_id_ids.keys():
        follower_id_ids[usr] = get_followers(usr)

In [24]:
#Get a network of RTs

In [25]:
rt_df = final[final.RT_userid > 0]

In [28]:
unique = list(set(rt_df.RT_userid))

In [31]:
rt_links = rt_df[['usr_id','RT_userid','usr_screen_name']]
rt_links.usr_id = [int(i) for i in rt_links.usr_id]
rt_links.RT_userid = [int(i) for i in rt_links.RT_userid]

C:\Users\Andre\Anaconda22\lib\site-packages\pandas\core\generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [44]:
rt_links[:5]

,usr_id,RT_userid,usr_screen_name
0,2237643924,2527838610,hotgamesandroid
1,2231098616,54343507,G2Play
2,2446741970,4727285215,qu33n_benis
4,196546017,4727285215,Tarumaturismo
5,180651830,440660492,AlphaProspector


In [37]:
nodes_names = {}
for i in copy.values:
    if i[0] not in nodes_names.keys():
        nodes_names[i[0]] = i[2]
    elif i[1] not in nodes_names.keys():
        nodes_names[i[1]] = i[2]

In [38]:
nodes_df = pd.DataFrame.from_dict(nodes_names,orient='index')

In [39]:
nodes_df['a'] = nodes_df.index
nodes_df.columns = ['Label','Id']
nodes_df = nodes_df[['Id','Label']]

In [42]:
nodes_df[:5]

,Id,Label
824392887911616512,824392887911616512,indiedevsunite
826692031628115968,826692031628115968,SZWLvo9OEuGPYwD
711375263628570624,711375263628570624,amixiusama
726733246021640192,726733246021640192,TwitchRTsBot
831353983700725760,831353983700725760,UnDcj6X7zFIxE8n


In [43]:
nodes_df.to_csv('nodes_labels')

How many nodes do we have?

In [47]:
len(nodes_df)

38511

Lets take a random sample of those nodes. Lets say 10%

In [ ]:
sample_rt = nodes_df.sample(3851)

copy.columns = ['Source','Target','Tag']
copy.to_csv('network')
sample_df = nodes_df.sample(3851)

Load "network" and "nodes_labels" into Gephi! Get nice graph :3